In [194]:
#####ベイジアン多変量回帰モデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import seaborn as sns
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [195]:
####任意の相関行列(分散共分散行列)を作成する関数####
##任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = (upper - lower) *rand(col*col) + lower   #相関係数の乱数ベクトルを作成
    rho = np.reshape(np.array(cov_vec), (col, col)) * np.tri(col)   #乱数ベクトルを下三角行列化
    Sigma = np.diag(np.diag(rho + rho.T) + 1) - (rho + rho.T)   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = scipy.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(eigen_val.shape[0]-1):
        if eigen_val[i] < 0:
            eigen_val[i] = (eigen_upper - eigen_lower) * rand(1) + eigen_lower
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(pow(np.diag(Sigma), 0.5)[:, np.newaxis], pow(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

In [196]:
##相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

In [197]:
####データの発生####
#データの設定
n = 100000   #サンプル数
k = 6   #応答変数数

##任意の分散共分散行列を発生させる
#パラメータを設定
col = 10
lower = -0.9
upper = 0.9
eigen_lower = 0
eigen_upper = 0.1
sigma_lower = 1.5
sigma_upper = 2.0

#相関行列を発生させる
Cor = CorM(col=col, lower=lower, upper=upper, eigen_lower=eigen_lower, eigen_upper=eigen_upper)
print(scipy.linalg.eigh(Cor)[0])   #正定値かどうか確認
print(np.round(Cor, 3))   #相関行列を確認

#分散共分散行列に変換
Cov = covmatrix(Cor=Cor, sigma_lower=sigma_lower, sigma_upper=sigma_upper)
print(scipy.linalg.eigh(Cov)[0])
print(np.round(Cov, 3))


##説明変数の発生
#連続変数の発生
cont = 8
X_cont = randn(n, cont)

#二値変数の発生
bin = 10
X_bin = np.zeros((n, bin))

for i in range(X_bin.shape[1]):
    p_bin = (0.7 - 0.4) * rand() + 0.4
    X_bin[:, i] = binomial(1, p_bin, n)

#多値変数の発生
multi = 8
p_multi = np.random.dirichlet(np.repeat(1.0, multi))
X_multi = multinomial(1, p_multi, n)

multi_sums = np.sum(X_multi, axis=0)
X_multi = np.delete(X_multi, multi_sums.argmin(), axis=1)   #冗長な変数を削除

#説明変数を結合
intercept = np.ones((n, 1))
X = np.concatenate((intercept, X_cont, X_bin, X_multi), axis=1)
col = X.shape[1]

[0.01212591 0.04858181 0.05148518 0.06296414 0.41395099 1.21760146
 1.49409515 1.68747876 2.38458124 2.62713536]
[[ 1.    -0.026 -0.526  0.309 -0.22  -0.239 -0.08  -0.338 -0.384 -0.243]
 [-0.026  1.    -0.363  0.248  0.433  0.306  0.168 -0.001  0.37  -0.511]
 [-0.526 -0.363  1.     0.248  0.064  0.511 -0.074  0.341 -0.084  0.179]
 [ 0.309  0.248  0.248  1.     0.291  0.42  -0.455 -0.023  0.061 -0.554]
 [-0.22   0.433  0.064  0.291  1.     0.051  0.216 -0.188 -0.088  0.212]
 [-0.239  0.306  0.511  0.42   0.051  1.     0.12   0.715 -0.13  -0.295]
 [-0.08   0.168 -0.074 -0.455  0.216  0.12   1.     0.042 -0.538  0.535]
 [-0.338 -0.001  0.341 -0.023 -0.188  0.715  0.042  1.    -0.14  -0.039]
 [-0.384  0.37  -0.084  0.061 -0.088 -0.13  -0.538 -0.14   1.    -0.572]
 [-0.243 -0.511  0.179 -0.554  0.212 -0.295  0.535 -0.039 -0.572  1.   ]]
[0.04244049 0.16678034 0.19240117 0.21680475 1.53090728 4.35677803
 5.00125325 6.26224381 8.92582477 9.15458385]
[[ 3.384 -0.092 -1.93   0.963 -0.806 -0.861

In [198]:
##応答変数の発生
#回帰パラメータの発生
BETA = np.zeros([X.shape[1], k])

for i in range(BETA.shape[1]):
    beta01 = uniform(1.5, 3.0, 1)
    beta02 = uniform(0, 1.0, cont)
    beta03 = uniform(-1.2, 1.6, (bin+multi-1))
    BETA[:, i] = np.concatenate((beta01, beta02, beta03), axis=0).T
BETAT = BETA

#分散共分散行列の発生
lower = -0.75
upper = 0.9
eigen_lower = 0
eigen_upper = 0.1
sigma_lower = 1.5
sigma_upper = 2.2

#相関行列を発生させる
Cor = CorM(col=k, lower=lower, upper=upper, eigen_lower=eigen_lower, eigen_upper=eigen_upper)
print(np.round(Cor, 3))   #相関行列を確認

#分散共分散行列に変換
Cov = covmatrix(Cor=Cor, sigma_lower=sigma_lower, sigma_upper=sigma_upper)
Covt = Cov
print(np.round(Cov, 3))

#回帰モデルの平均構造を設定
Z = np.dot(X, BETA)

#平均構造に多変量正規分布の誤差を加える
Y = np.zeros((n, k))

for i in range(Y.shape[0]):
    Y[i, ] = np.random.multivariate_normal(Z[i, :], Cov)

#発生させた応答変数の集計
print(np.average(Y, axis=1))
print(np.corrcoef(Y))

#scatter_matrix(pd.DataFrame(Y), diagonal="kde", color="k", alpha=0.3)
Y_pd = pd.DataFrame(Y)
scatter_matrix(Y_pd, diagonal='kde', color='k', alpha=0.3)
plt.show()

[[ 1.    -0.693 -0.461  0.201  0.456  0.435]
 [-0.693  1.     0.664 -0.703  0.177 -0.294]
 [-0.461  0.664  1.    -0.28  -0.085  0.434]
 [ 0.201 -0.703 -0.28   1.    -0.725  0.453]
 [ 0.456  0.177 -0.085 -0.725  1.    -0.148]
 [ 0.435 -0.294  0.434  0.453 -0.148  1.   ]]
[[ 2.87  -1.956 -1.665  0.667  1.465  1.213]
 [-1.956  2.78   2.363 -2.295  0.559 -0.807]
 [-1.665  2.363  4.55  -1.168 -0.342  1.524]
 [ 0.667 -2.295 -1.168  3.83  -2.692  1.459]
 [ 1.465  0.559 -0.342 -2.692  3.602 -0.463]
 [ 1.213 -0.807  1.524  1.459 -0.463  2.711]]
[4.82558831 4.35520663 3.35715477 ... 2.31940226 6.62058176 4.46680642]


MemoryError: 

In [199]:
#####ギブスサンプリングで多変量回帰モデルを推定#####
##多変量正規分布の密度関数
def dmv(x, mu, Cov, k):
    er = x - mu
    Cov_inv = np.linalg.inv(Cov) 
    LLo = 1 / (np.sqrt(pow((2 * np.pi), k) * np.linalg.det(Cov))) * np.exp(-1/2 * np.dot(np.dot(er, Cov_inv) *er, np.ones(k)))
    return(LLo)

In [200]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 10000   #サンプリング回数
keep = 4   #4回に1回の割合でサンプリング結果を保存
disp = 200
sbeta = 1.5

In [201]:
##事前分布の設定
Deltabar = np.zeros((col, k))
ADelta = 0.01 * np.diag(np.ones(col))
nu = k + 1
V = nu * np.diag(np.ones(k))

In [202]:
##初期値の設定
beta = np.zeros((col, k))
Cov = np.diag(np.ones(k))
np.sum(np.log(dmv(Y, np.dot(X, beta), Cov, k)))   #初期値での対数尤度

-8927636.0201689

In [203]:
##定数の設定
XX = np.dot(X.T, X)
XY = np.dot(X.T, Y)
inv_XXV = np.linalg.inv(np.dot(X.T, X) + ADelta)

In [204]:
##パラメータの格納用配列
BETA = np.zeros((col, k, int(R/keep)))
COV = np.zeros((k, k, int(R/keep)))
LL = np.zeros((int(R/keep)))

In [205]:
####ギブスサンプラーでパラメータをサンプリング####
for rp in range(R):
    ##多変量正規分布から回帰行列をサンプリング
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)

    
    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        BETA[:, :, mkeep] = beta
        COV[:, :, mkeep] = Cov

    if rp%disp==0:
        #対数尤度の更新
        LL = np.sum(np.log(dmv(Y, mu, Cov, k)))

        #サンプリング結果を確認
        print(rp)
        print(LL)

0
-730958.1587145946
200
-729742.208586841
400
-729723.8941236418
600
-729729.6083736494
800
-729726.0353476817
1000
-729724.2977674415
1200
-729724.2897091351
1400
-729720.360285927
1600
-729737.7322277434
1800
-729739.966223167
2000
-729747.1945274046
2200
-729720.5157470453
2400
-729741.3068941057
2600
-729732.4856409895
2800
-729744.8592268145
3000
-729743.0307464917
3200
-729720.7820367438
3400
-729729.9752413115
3600
-729726.9085562738
3800
-729732.3587619149
4000
-729743.1155158952
4200
-729739.1897177391
4400
-729737.1182540002
4600
-729733.395259307
4800
-729726.727293381
5000
-729742.9075687827
5200
-729707.7827085743
5400
-729725.1981666194
5600
-729744.7223866575
5800
-729731.5061801315
6000
-729722.8151619958
6200
-729722.0669739401
6400
-729741.8303358781
6600
-729747.9560273441
6800
-729739.5351086125
7000
-729739.2466248085
7200
-729728.6623208985
7400
-729744.954133098
7600
-729729.5037908634
7800
-729729.3763233989
8000
-729724.1210502834
8200
-729744.6610098436
8400
